In [1]:
import numpy as np
import pandas as pd

accidentes = pd.read_csv('./data/Siniestros/Siniestros_Viales_Bogota_Accidente.csv', low_memory=False)
lesionados = pd.read_csv('./data/Siniestros/Siniestros_Viales_Bogota_Lesionados.csv', index_col='FORMULARIO', low_memory=False)
muertos = pd.read_csv('./data/Siniestros/Siniestros_Viales_Bogota_Muertos.csv', index_col='FORMULARIO', low_memory=False)
vehiculos = pd.read_csv('./data/Siniestros/AnA1lisis_Siniestralidad_vehiculos.csv', index_col='FORMULARIO', low_memory=False)
causas = pd.read_csv('./data/Siniestros/AnA1lisis_Siniestralidad_causas.csv', index_col='FORMULARIO', low_memory=False)

In [2]:
columns=["X", "Y","OBJECTID","HORA_OCURRENCIA_ACC","ANO_OCURRENCIA_ACC","MES_OCURRENCIA_ACC",
         "DIA_OCURRENCIA_ACC","LOCALIDAD", "LATITUD","LONGITUD","FECHA"]

lesionados.drop(columns, axis=1, inplace=True)
lesionados.replace({'': None}, inplace=True) 
lesionados.replace({np.nan: None}, inplace=True)
lesionados.fillna({0: None}, inplace=True)

In [3]:
lesionados_col = lesionados.groupby('FORMULARIO', level=0).apply(lambda x: x.to_dict(orient='records')).to_frame()
lesionados_col.rename(columns={0:"LESIONADOS"}, inplace=True, errors="raise")

In [4]:
lesionados_accidentes = accidentes.join(lesionados_col, on='FORMULARIO')

In [5]:
columns=["X", "Y","OBJECTID","HORA_OCURRENCIA_ACC","ANO_OCURRENCIA_ACC","MES_OCURRENCIA_ACC",
         "DIA_OCURRENCIA_ACC","LOCALIDAD", "LATITUD","LONGITUD","FECHA"]

muertos.drop(columns, axis=1, inplace=True)

muertos['MUERTE_POSTERIOR'] = muertos['MUERTE_POSTERIOR'].map({'N':False, 'S': True})
muertos['FECHA_POSTERIOR_MUERTE']=pd.to_datetime(muertos['FECHA_POSTERIOR_MUERTE'])
muertos.replace({'': None}, inplace=True) 
muertos.replace({np.nan: None}, inplace=True)
muertos.fillna({0: None}, inplace=True)

In [6]:
muertos_col = muertos.groupby('FORMULARIO', level=0).apply(lambda x: x.to_dict(orient='records')).to_frame()
muertos_col.rename(columns={0:"MUERTOS"}, inplace=True, errors="raise")

In [7]:
final = lesionados_accidentes.join(muertos_col, on='FORMULARIO')

In [8]:
dia_semana={0:'LUNES', 1:'MARTES', 2:'MIERCOLES', 3:'JUEVES', 4:'VIERNES', 5:'SABADO', 6:'DOMINGO'}
mes={1:'ENERO', 2:'FEBRERO', 3:'MARZO', 4:'ABRIL', 5:'MAYO', 6:'JUNIO',
     7:'JULIO', 8:'AGOSTO', 9:'SEPTIEMBRE', 10:'OCTUBRE', 11:'NOVIEMBRE', 12:'DICIEMBRE'}

final['FECHA_HORA'] = pd.to_datetime(final['FECHA']+" "+final['HORA_OCURRENCIA_ACC'],  dayfirst=True)
final['FECHA_HORA_ACC']=pd.to_datetime(final['FECHA_HORA_ACC'])
final['HORA'] = final['FECHA_HORA'].dt.hour
final['DIA'] = final['FECHA_HORA'].dt.day
final['DIA_SEMANA'] = final['FECHA_HORA'].dt.dayofweek.map(dia_semana)
final['MES'] = final['FECHA_HORA'].dt.month.map(mes)
final['ANO'] = final['FECHA_HORA'].dt.year

columns=["OBJECTID", "X", "Y", "HORA_OCURRENCIA_ACC", "ANO_OCURRENCIA_ACC", "MES_OCURRENCIA_ACC", "DIA_OCURRENCIA_ACC"]

final.drop(columns, axis=1, inplace=True)
final.dropna(subset="FECHA", inplace=True)

In [10]:
columns=["OBJECTID", "CODIGO_ACCIDENTE", "CODIGO", "TIPO"]
causas.drop(columns, axis=1, inplace=True)
causas.rename(columns={"NOMBRE":"NOMBRE_CAUSA"}, inplace=True, errors="raise")

In [11]:
vehiculos = vehiculos.merge(causas, how="left", on=['FORMULARIO', 'CODIGO_VEHICULO'])

In [12]:
vehiculos.drop(["OBJECTID"], axis=1, inplace=True)
vehiculos["PLACA"] = vehiculos["PLACA"].str.strip()
vehiculos['ENFUGA'] = vehiculos['ENFUGA'].map({'N':False, 'S': True})

values = {"PLACA": "", "CLASE": "", "MODALIDAD": "", "ENFUGA": "", "NOMBRE_CAUSA": "", "TIPO_CAUSA":""}

vehiculos.fillna(value=values, inplace=True)
vehiculos.replace({'': None}, inplace=True)
vehiculos.replace({np.nan: None}, inplace=True)
vehiculos.fillna({0: None}, inplace=True)

In [ ]:
import pygeohash as gh

vehiculos_col = vehiculos.groupby('FORMULARIO', level=0).apply(lambda x: x.to_dict(orient='records')).to_frame()
vehiculos_col.rename(columns={0:"VEHICULOS"}, inplace=True, errors="raise")

final = final.join(vehiculos_col, on='FORMULARIO')

final["GEO"] = final['LATITUD'].astype(str)+','+final['LONGITUD'].astype(str)
final["GEOHASH"] = final.apply(lambda x: gh.encode(x.LATITUD, x.LONGITUD, precision=6), axis=1)

In [ ]:
values = {"HORA": "", "MES": "", "DIA": "", "DIA_SEMANA": "", "GRAVEDAD": "", "CLASE_ACC":"", "LOCALIDAD": "",
          "FECHA_HORA_ACC": "", "CONDICION": "", "GENERO":"", "MUERTE_POSTERIOR":"", "FECHA_POSTERIOR_MUERTE":""}

final.fillna(value=values, inplace=True)
final.replace({'': None}, inplace=True) 
final.replace({np.nan: None}, inplace=True)
final.fillna({0: None}, inplace=True)

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

NODES = ["https://myelasticinstance01:9200","https://myelasticinstance02:9200"]
es_client = Elasticsearch(
    NODES,
    #verify_certs=False,
    #ssl_show_warn=False,
    basic_auth=("myawesomename", "mysupersecretpassword")
)

es_client.info()

In [ ]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        doc = {
            "_index" : "movilidad_accidentes",
            "_id" : index,
            "_source" : document.to_dict()
        }
        yield doc
        

In [ ]:
for success, info in helpers.parallel_bulk(es_client, doc_generator(final)):
    if not success:
        print('A document failed:', info)

One little aggregation to get you started making your very own ML Algorithm

In [ ]:
from sklearn.preprocessing import LabelEncoder

data = final.groupby([pd.Grouper(key='FECHA_HORA', freq='1d'), 'GEOHASH']).count().reset_index().sort_values(['FECHA_HORA','GEOHASH'])
display(data)